# Analyse des Dossiers de Patients Atteints de Cancer

Ce notebook contient une analyse complète des dossiers de patients atteints de cancer, incluant :
- Chargement des données
- Prétraitement et nettoyage des données
- Vectorisation TF-IDF
- Entraînement d'un modèle Random Forest
- Évaluation du modèle
- Visualisations des résultats

In [ ]:
# Installation des packages nécessaires
%pip install pandas numpy scikit-learn matplotlib seaborn

# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Pour afficher les graphiques dans le notebook
%matplotlib inline

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Pour afficher les graphiques dans le notebook
%matplotlib inline

**Important** : Après l'installation des packages :
1. Redémarrez le kernel du notebook (Menu Kernel -> Restart)
2. Puis réexécutez toutes les cellules dans l'ordre

In [ ]:
# Chargement des données depuis le fichier local
df = pd.read_csv('china_cancer_patients_synthetic.csv')

# Affichage des premières lignes et informations sur le dataset
print("\nAperçu des données :")
print(df.head())
print("\nInformations sur le dataset :")
print(df.info())

In [ ]:
# Nettoyage des données
def clean_text(text):
    if isinstance(text, str):
        # Conversion en minuscules
        text = text.lower()
        # Suppression des caractères spéciaux
        text = re.sub(r'[^\w\s]', '', text)
        # Suppression des espaces multiples
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ''

# Application du nettoyage aux colonnes textuelles
# Note: Ajustez les noms des colonnes en fonction de votre dataset
text_columns = ['description']  # Modifiez selon vos colonnes
for col in text_columns:
    if col in df.columns:
        df[f'{col}_clean'] = df[col].apply(clean_text)

# Prétraitement des données

# Traitement des valeurs manquantes
print("Valeurs manquantes par colonne :")
print(df.isnull().sum())

# Remplacement des valeurs manquantes
df['AlcoholUse'] = df['AlcoholUse'].fillna('Unknown')
df['GeneticMutation'] = df['GeneticMutation'].fillna('None')
df['Comorbidities'] = df['Comorbidities'].fillna('None')

# Encodage des variables catégorielles
cat_columns = ['Gender', 'Province', 'Ethnicity', 'TumorType', 'CancerStage', 
               'Metastasis', 'TreatmentType', 'SmokingStatus', 'AlcoholUse', 
               'GeneticMutation']

# Utilisation de get_dummies pour l'encodage one-hot
df_encoded = pd.get_dummies(df[cat_columns])

# Ajout des colonnes numériques
num_columns = ['Age', 'TumorSize', 'ChemotherapySessions', 'RadiationSessions', 'FollowUpMonths']
df_final = pd.concat([df_encoded, df[num_columns]], axis=1)

print("\nDimensions des données après prétraitement:", df_final.shape)

In [ ]:
# Préparation des données pour le modèle
X = df_final  # Toutes les colonnes encodées
y = (df['SurvivalStatus'] == 'Deceased').astype(int)  # 1 pour 'Deceased', 0 pour 'Alive'

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensions des données d'entraînement:", X_train.shape)
print("Dimensions des données de test:", X_test.shape)
print("\nDistribution de la cible:")
print(y.value_counts(normalize=True))

# Préparation des données pour le modèle
X = df_final  # Toutes les colonnes encodées
y = (df['SurvivalStatus'] == 'Deceased').astype(int)  # 1 pour 'Deceased', 0 pour 'Alive'

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensions des données d'entraînement:", X_train.shape)
print("Dimensions des données de test:", X_test.shape)

In [ ]:
# Création et entraînement du modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = rf_model.predict(X_test)

In [ ]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Métriques d'évaluation :")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

In [ ]:
# Matrice de confusion
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de Confusion')
plt.xlabel('Prédictions (0: Vivant, 1: Décédé)')
plt.ylabel('Valeurs Réelles (0: Vivant, 1: Décédé)')
plt.show()

# Importance des caractéristiques
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
})

# Affichage des 20 caractéristiques les plus importantes
plt.figure(figsize=(12, 8))
top_features = feature_importance.nlargest(20, 'importance')
sns.barplot(data=top_features, x='importance', y='feature')
plt.title('Top 20 des Caractéristiques les Plus Importantes')
plt.xlabel('Importance')
plt.ylabel('Caractéristique')
plt.tight_layout()
plt.show()